<a href="https://colab.research.google.com/github/gianca1990/pdf_extraction/blob/main/berami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google import genai
from google.genai import types
import pathlib
import httpx
import csv
!pip install pymupdf
import fitz
import json

In [2]:
# Initialize the Google GenAI client with the provided API key
client = genai.Client(api_key="")

In [33]:
# Filepath to the local PDF file
filepath = pathlib.Path("/TO_LOAD.pdf")

In [34]:
# Read the PDF file as bytes
pdf_bytes = filepath.read_bytes()

In [35]:
def extract_checked_checkboxes(pdf_path):

    doc = fitz.open(pdf_path)
    checkboxes = []

    for page in doc:
        for widget in page.widgets():
            if widget.field_type == 2 and widget.field_value == "On":
                checkboxes.append({"name": widget.field_name, "state": widget.field_value})

    prompt_data = {
        "checkboxes": checkboxes
    }

    return json.dumps(prompt_data, indent=4)

In [57]:
pdf_path = "/TO_LOAD.pdf"
extract_checked_checkboxes(pdf_path)

'{\n    "checkboxes": [\n        {\n            "name": "m\\u00e4nnlich",\n            "state": "On"\n        },\n        {\n            "name": "Wohnsitz in Deutschland",\n            "state": "On"\n        },\n        {\n            "name": "RS selbst",\n            "state": "On"\n        },\n        {\n            "name": "soziales Umfeld",\n            "state": "On"\n        },\n        {\n            "name": "weiblich",\n            "state": "On"\n        },\n        {\n            "name": "Wohnsitz au\\u00dferhalb Deutschlands",\n            "state": "On"\n        },\n        {\n            "name": "divers",\n            "state": "On"\n        },\n        {\n            "name": "keine Angabe zum Geschlecht",\n            "state": "On"\n        }\n    ]\n}'

In [67]:
def extract_form_fields(pdf_path):
    """
    Extrahiert alle Formularfelder (Checkboxen und Textfelder) aus einem PDF.
    Gibt eine JSON-Ausgabe mit den Namen und Werten der Felder zurück.
    """
    doc = fitz.open(pdf_path)
    form_data = []

    for page in doc:
        for widget in page.widgets():  # Alle Formularfelder auf der Seite durchgehen
            field_name = widget.field_name  # Name des Feldes
            field_value = widget.field_value  # Wert des Feldes

            # Überprüfen, welcher Feldtyp vorliegt
            if widget.field_type == 2:  # 2 = Checkbox
                field_type = "Checkbox"
                field_value = "On" if field_value == "On" else "Off"
            elif widget.field_type == 7:  # 1 = Textfeld (Inputfeld)
                field_type = "Text Input"
            else:
                field_type = "Unknown"  # Für andere unbekannte Feldtypen

            # Formularfeld in die Liste einfügen
            form_data.append({
                "name": field_name,
                "type": field_type,
                "value": field_value if field_value else "No value"
            })

    # JSON-Ausgabe erstellen
    json_output = json.dumps({"form_fields": form_data}, indent=4)
    return json_output

In [68]:
pdf_path = "/TO_LOAD.pdf"
extract_form_fields(pdf_path)

'{\n    "form_fields": [\n        {\n            "name": "Erwerbst\\u00e4tigkeit im Ausland",\n            "type": "Checkbox",\n            "value": "Off"\n        },\n        {\n            "name": "Erwerbst\\u00e4tigkeit in Deutschland",\n            "type": "Checkbox",\n            "value": "Off"\n        },\n        {\n            "name": "beitragspflichtig besch\\u00e4ftigt",\n            "type": "Checkbox",\n            "value": "Off"\n        },\n        {\n            "name": "Name RS",\n            "type": "Text Input",\n            "value": "Mustermann"\n        },\n        {\n            "name": "E-Mail RS",\n            "type": "Text Input",\n            "value": "arielaw9@gmail.com"\n        },\n        {\n            "name": "Telefonnummer RS",\n            "type": "Text Input",\n            "value": "016093455298"\n        },\n        {\n            "name": "Geburtsjahr",\n            "type": "Text Input",\n            "value": "1963"\n        },\n        {\n            

## Header

In [77]:
# IMPORTANT: There are single and multiple choices. Which are single and which are multiple choices? How do you deal with multiple choices? What does the database expect at this point?
# Prompt for extracting specific data from the PDF/JSON
prompt = """
Extract the required "value" for the names from the provided JSON.
""" + extract_form_fields(pdf_path) + """
The following list corresponds to the values that need to be extracted from the provided JSON:
"name": "DATUM der Beratung";
"name": "NAME BERATER*IN";
Extract the "name" value from the JSON where "value" is set to "On", but only between the following names: "Telefonberatung" and "schriftliche Einwilligungserklärung liegt vor".

Output only the marked text. Format the output as CSV with the headers:
Datum;
Berater*in;
Beratung.

Use semicolons as separators.
If a piece of information is missing, leave the field blank.
Only extract the dataset in the CSV format.
Do not include any additional information, explanations, or text."""

In [78]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [79]:
# Print the response text for debugging or confirmation
print(response.text)

Datum;Berater*in;Beratung
30.07.24;DV;




## Persönliche Daten Ratsuchend*er (RS*)

In [53]:
# IMPORTANT: The extraction of the information for "Geschlecht:" seems to be difficult.
# Prompt for extracting specific data from the PDF
prompt = """
Extract the required information from either the PDF or the provided JSON.
If the information is available in the JSON, use it instead of extracting it from the PDF.
The JSON always takes priority over the PDF. Only extract data from the PDF if it is missing from the JSON.
""" + extract_checked_checkboxes(pdf_path) + """
Field 1: Extract the "name" values from the JSON where "state" is set to "On", but only for the following names: "RS selbst" and "soziales Umfeld";
Next to "Name:";


Output only the marked text. Format the output as CSV with the headers:
Wer nimmt die Beratung wahr?;
Name;


Use semicolons as separators.
If a piece of information is missing, leave the field blank.
Only extract the dataset in the CSV format.
Do not include any additional information, explanations, or text."""

In [54]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [55]:
# Print the response text for debugging or confirmation
print(response.text)

Wer nimmt die Beratung wahr?;Name;
Person mit ausl. Abschl.;;
selbst;soziales Umfeld;


## Berufliche Situation und Aufenthaltsstatus

In [ ]:
# IMPORTANT: Each field extraction must be provided with the addition "If the checkbox is not checked, leave the field blank.".
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Ist der/die RS erwerbstätig". The options are 'ja, im Ausland', 'nein', 'keine Angabe', 'ja, in Deutschland' and 'nein, jedoch im Aus-/Weiterbildung/Qualifizierung';
Extract the content of the fields that are marked with a checked checkbox (✔ or ☑) next to the heading "Art der Erwerbstätigkeit". The options are 'beitragspflichtig beschäftigt', 'geringfügig beschäftigt' and 'selbstständig'. If the checkbox is not checked, leave the field blank.

Output only the marked text. Format the output as CSV with the headers:
Ist der/die RS erwerbstätig;
Art der Erwerbstätigkeit.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [ ]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [ ]:
# Print the response text for debugging or confirmation
print(response.text)

```csv
Ist der/die RS erwerbstätig;Art der Erwerbstätigkeit
nein;
```



## Bezug & Status

In [ ]:
# IMPORTANT: Each field extraction must be provided with the addition "If the checkbox is not checked, leave the field blank.".
# Prompt for extracting specific data from the PDF
prompt = """
Extract the following information from the attached PDF and return it as CSV:
Extract the content of the fields that are marked with a checked checkbox. But only between he following options: 'ohne Leistungsbezug', 'ohne Leistungsbezug, jedoch arbeitssuchend gemeldet', 'mit (ergänzendem) SGB II-Leistungsbezug', 'mit SGB III-Leistungsbezug', 'mit SGB III- und SGB II-Leistungsbezug', 'mit (ergänzendem) Asylbewerberleistungsbezug' and 'keine Angabe'. If the checkbox is not checked, leave the field blank.;
Extract the content of the fields that are marked with a checked checkbox.
But only between he following options:
'Aufenthaltserlaubnis zur Arbeitsplatzsuche (§20 AufenthG)',
'Aufenthaltserlaubnis für in anderen Mitgliedsstaaten der Europäischen Unionlangfristig Aufenthaltsberechtigte (§38a AufenthG)',
'Aufenthaltsgestattung (§55 Abs. 1 AsylVfG)',
'Aufenthalt aus familiären Gründen (§27-36 AufenthG)',
'Aufenthalt aus völkerrechtlichen, humanitären oder politischen Gründen(22-26, 104a, 104b AufenthG)',
'Aufenthalt zum Zweck der Ausbildung (§16a und 17 AufenthG)',
'Aufenthalt zum Zweck der Erwerbstätigkeit (§18, 18a, 20, 21 AufenthG)',
'Aufenthalt zum Zweck einer Anpassungsqualifizierung oder einer Kenntnisprüfung16d AufenthG)',
'Blaue Karte EU (§18b, Abs. 2 AufenthG)',
'Duldung (§60a, 60b, 60c, 60d AufenthG)',
'Niederlassungserlaubnis (§9 AufenthG)',
'Staatsbürger-/in EU/EWR/Schweiz oder Freizügigkeit 2-5, §§12-13, §15 FreizügG/EU, §28 AufenthV)☐Visum (§6 AufenthG)' and
'kein Aufenthaltstitel, da Wohnsitz im Ausland'
If the checkbox is not checked, leave the field blank.

Output only the marked text. Format the output as CSV with the headers:
Leistungsbezug;
Art der Erwerbstätigkeit.

Use semicolons as separators. If a piece of information is missing, leave the field blank. Only extract the dataset in the CSV format. Do not include any additional information, explanations, or text."""

In [ ]:
# Send the PDF and prompt to Google GenAI for content generation
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=filepath.read_bytes(),
        mime_type='application/pdf',
      ),
      prompt])

In [ ]:
# Print the response text for debugging or confirmation
print(response.text)

```csv
Leistungsbezug;Art der Erwerbstätigkeit
;beitragspflichtig beschäftigt
```



In [ ]:
# Define the output CSV filename
filename = "/content/EIGENE_DATEIEN/DATA.csv"

In [11]:
def extract_checked_checkboxes(pdf_path):
    """
    Extrahiert und gibt alle aktivierten Checkboxen aus einem PDF-Dokument aus.

    Parameter:
    pdf_path (str): Der Pfad zur PDF-Datei.

    Ablauf:
    1. Öffnet das PDF-Dokument mit PyMuPDF (fitz).
    2. Durchläuft alle Seiten des Dokuments.
    3. Sucht nach interaktiven Feldern (Widgets) auf jeder Seite.
    4. Prüft, ob das Widget vom Typ "Checkbox" ist.
    5. Wenn die Checkbox aktiviert ist ("On"), wird ihr Name und Wert ausgegeben.
    """

    doc = fitz.open(pdf_path)  # PDF-Dokument öffnen

    for page in doc:  # Jede Seite im Dokument durchlaufen
        for widget in page.widgets():  # Alle interaktiven Felder der Seite durchgehen
            if widget.field_type == 2:  # Prüfen, ob es sich um eine Checkbox handelt (2 = Checkbox)
                value = widget.field_value  # Wert der Checkbox abrufen
                if value == "On":  # Nur aktivierte Checkboxen weiterverarbeiten
                    print(f"Feldname: '{widget.field_name}' | Wert: '{value}'")  # Name und Wert ausgeben

In [12]:
pdf_path = "/TO_LOAD.pdf"
extract_checked_checkboxes(pdf_path)

Feldname: 'Telefonberatung' | Wert: 'On'
Feldname: 'schriftliche Einwilligungserklärung liegt vor' | Wert: 'On'
